![PUC Logo](http://www.inf.puc-rio.br/~inf1005/images/logo_di.png)

# Segundo Trabalho de Estruturas Discretas

>Professor: Prof. Marcus Vinicius S. Poggi de Aragão
- Este trabalho prático consiste em desenvolver códigos para diferentes algoritmos e estruturas de dados para resolver os problemas descritos e, principalmente, analisar o desempenho das implementações destes algoritmos com respeito ao tempo de CPU.

Integrantes do grupo:
- Gabriel Barbosa Diniz - 1511211
- Lucas Rodrigues - 1510848
- Mateus Ribeiro de Castro - 1213068

## Primeira Questão (Teorema 1)

> Dado um teorema do tipo: Sabe-se encontrar a árvore de peso máximo de G=(V,E) que contém o vértice 1 e possui K vértices; deseja-se encontrar o algoritmo correspondente que prove o teorema e teste o algoritmo em instâncias pequenas dos arquivos de dados fornecidos.

### (I) Implementação Python com Testes:

## Primeira Questão (Teorema 1 - BÔNUS)

> Repita os itens anteriores para o teorema do tipo: Sabe-se encontrar a floresta de peso mínimo de G = (V, E) onde os componentes conexos possuem pelo menos K vértices.

### (I) Implementação Python com Testes:

## Segunda Questão (Teorema 2)

> Dado um teorema do tipo: Sabe-se determinar o prêmio máximo que o rei consegue coletar estando na posição (i,j) com q unidades restantes para consumir; apresente um algoritmo para determinar o caminho que o rei deve fazer para obter o maior total possível em prêmios e teste para um arquivo de dados com a estrutura apresentada.

### (I) Implementação Python com Testes:

## Terceira Questão (Teorema 3)

> Considere um text definido pelo vetor de caracteres T[1..n]. Deseja-se determinar todas as ocorrências do string s[1..m] em T. Desenvolva algoritmos com este objetivo que atendam as condições determinadas...

### (I) Implementação Python com Testes: